<a href="https://colab.research.google.com/github/trupti-04oct/repo/blob/main/ML_Model_Regression_using_MLlib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["PATH"] += ":/usr/lib/jvm/java-11-openjdk-amd64/bin"


In [11]:
!apt-get update -y
!apt-get install openjdk-11-jdk -y


Hit:1 https://cli.github.com/packages stable InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading packag

In [15]:
!wget -q https://downloads.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz


In [16]:
!tar -xzf spark-3.4.1-bin-hadoop3.tgz
!mv spark-3.4.1-bin-hadoop3 /usr/local/spark


gzip: stdin: unexpected end of file
tar: Unexpected EOF in archive
tar: Unexpected EOF in archive
tar: Error is not recoverable: exiting now


In [17]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [21]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("films") \
    .master("local[*]") \
    .getOrCreate()

sc = spark.sparkContext


In [24]:
import pandas as pd

df = pd.read_excel('/content/Boston.csv.xlsx')
df.to_csv('/content/Boston.csv', index=False)


In [25]:
house_df = spark.read.csv('/content/Boston.csv', header=True, inferSchema=True)
house_df.show()


+----------+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|Unnamed: 0|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio| black|lstat|medv|
+----------+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|         1|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|         2|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|         3|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|         4|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|         5|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|         6|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|         7|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|         8|0.14455|12.5| 7.87|   0|0.52

In [26]:
## Printing schema
house_df.printSchema()


root
 |-- Unnamed: 0: integer (nullable = true)
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- black: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [27]:
## Descriptive analysis
house_df.toPandas()

,Unnamed: 0,crim,zn,indus,chas,nox,rm,age,dis,rad,tax,ptratio,black,lstat,medv
0,1,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,2,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,3,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,4,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,5,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,502,0.06263,0.0,11.93,0,0.573,6.593,69.1,2.4786,1,273,21.0,391.99,9.67,22.4
502,503,0.04527,0.0,11.93,0,0.573,6.120,76.7,2.2875,1,273,21.0,396.90,9.08,20.6
503,504,0.06076,0.0,11.93,0,0.573,6.976,91.0,2.1675,1,273,21.0,396.90,5.64,23.9
504,505,0.10959,0.0,11.93,0,0.573,6.794,89.3,2.3889,1,273,21.0,393.45,6.48,22.0


In [28]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'black', 'lstat'], outputCol = 'features')
vhouse_df = vectorAssembler.transform(house_df)
vhouse_df = vhouse_df.select(['features', 'medv'])
vhouse_df.show(3)

+--------------------+----+
|            features|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
+--------------------+----+
only showing top 3 rows



In [29]:
splits = vhouse_df.randomSplit([0.7, 0.3])
train_df = splits[0]
test_df = splits[1]
#train_df,test_df=vhouse_df.randomSplit([0.7,0.3])

In [30]:
from pyspark.ml.regression import LinearRegression
lr = LinearRegression(featuresCol = 'features', labelCol='medv', maxIter=10)
lr_model = lr.fit(train_df)
print("Coefficients: " + str(lr_model.coefficients))
print("Intercept: " + str(lr_model.intercept))

Coefficients: [-0.052786144136872576,0.054309742957239934,-0.0047690223987326506,1.0450508458358072,-16.521478391353646,4.373054642242128,-0.004033376195459216,-1.4692965651376844,0.31037911910345334,-0.013362838507801322,-0.9701835420946042,0.011140494543817098,-0.4718969249060562]
Intercept: 31.927250368764764


In [31]:
trainingSummary = lr_model.summary
print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
print("r2: %f" % trainingSummary.r2)

RMSE: 4.463705
r2: 0.763040


In [32]:
lr_predictions = lr_model.transform(test_df)
lr_predictions.select("prediction","medv","features").show(5)
from pyspark.ml.evaluation import RegressionEvaluator
lr_evaluator = RegressionEvaluator(predictionCol="prediction", \
                 labelCol="medv",metricName="r2")
print("R Squared (R2) on test data = %g" % lr_evaluator.evaluate(lr_predictions))

+------------------+----+--------------------+
|        prediction|medv|            features|
+------------------+----+--------------------+
|32.718767024456405|32.2|[0.00906,90.0,2.9...|
|32.291159349277116|35.4|[0.01311,90.0,1.2...|
|25.832772890314907|23.1|[0.0187,85.0,4.15...|
|23.851959793124273|33.0|[0.01951,17.5,1.3...|
|38.189614383256554|42.3|[0.02177,82.5,2.0...|
+------------------+----+--------------------+
only showing top 5 rows

R Squared (R2) on test data = 0.669256


In [33]:
print("numIterations: %d" % trainingSummary.totalIterations)
print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
trainingSummary.residuals.show()


numIterations: 0
objectiveHistory: [0.0]
+-------------------+
|          residuals|
+-------------------+
| -6.068141835815386|
|-5.8015721030364595|
| 1.7722840373212208|
|   3.26069091748618|
|  7.839661304082661|
| -2.366979603418514|
| -2.875457172813622|
|-3.7458282242164636|
|  5.485488634965321|
|  5.482727397446723|
| 3.8908695415917336|
| 1.3313554320029901|
|-0.7007388984149472|
|  5.338756605606605|
|-0.6878972119942084|
|-1.6470571333310104|
| -5.517624470539296|
|-4.1967154300701175|
|-3.2327345934187477|
| -1.108573702169675|
+-------------------+
only showing top 20 rows



In [34]:
predictions = lr_model.transform(test_df)
predictions.select("prediction","medv","features").show()

+------------------+----+--------------------+
|        prediction|medv|            features|
+------------------+----+--------------------+
|32.718767024456405|32.2|[0.00906,90.0,2.9...|
|32.291159349277116|35.4|[0.01311,90.0,1.2...|
|25.832772890314907|23.1|[0.0187,85.0,4.15...|
|23.851959793124273|33.0|[0.01951,17.5,1.3...|
|38.189614383256554|42.3|[0.02177,82.5,2.0...|
|30.611594965570397|34.7|[0.02729,0.0,7.07...|
| 22.57656801221118|26.6|[0.02899,40.0,1.2...|
|25.077517362316833|28.7|[0.02985,0.0,2.18...|
|28.690197602702657|33.4|[0.03237,0.0,2.18...|
|30.735171824402983|23.5|[0.03584,80.0,3.3...|
| 32.38105261793354|27.9|[0.03615,80.0,4.9...|
|27.229047820735133|23.2|[0.03871,52.5,5.3...|
|20.461325712355013|21.1|[0.03961,0.0,5.19...|
| 37.07420339368413|33.3|[0.04011,80.0,1.5...|
| 28.54708522910087|28.0|[0.04113,25.0,4.8...|
|31.361285957607382|24.8|[0.04417,70.0,2.2...|
|27.417462215384607|22.3|[0.0459,52.5,5.32...|
|27.036484303982828|22.6|[0.04684,0.0,3.41...|
|24.054586003

## Decision tree regression

In [35]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'medv')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="rmse")
rmse = dt_evaluator.evaluate(dt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 4.52476


In [36]:
from pyspark.ml.regression import DecisionTreeRegressor
dt = DecisionTreeRegressor(featuresCol ='features', labelCol = 'medv')
dt_model = dt.fit(train_df)
dt_predictions = dt_model.transform(test_df)
dt_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="r2")
r2 = dt_evaluator.evaluate(dt_predictions)
print("R2 on test data = %g" % r2)

R2 on test data = 0.75966


In [37]:
lr_evaluator.evaluate(dt_predictions)

0.7596597002776723

In [39]:
dt_model.featureImportances

SparseVector(13, {0: 0.0912, 1: 0.0029, 2: 0.0152, 4: 0.0072, 5: 0.6367, 6: 0.002, 7: 0.0011, 8: 0.0196, 9: 0.0085, 10: 0.0059, 12: 0.2099})

In [40]:
house_df.take(1)

[Row(Unnamed: 0=1, crim=0.00632, zn=18.0, indus=2.31, chas=0, nox=0.538, rm=6.575, age=65.2, dis=4.09, rad=1, tax=296, ptratio=15.3, black=396.9, lstat=4.98, medv=24.0)]

## Gradient-boosted tree regression

In [41]:
from pyspark.ml.regression import GBTRegressor
gbt = GBTRegressor(featuresCol = 'features', labelCol = 'medv', maxIter=10)
gbt_model = gbt.fit(train_df)
gbt_predictions = gbt_model.transform(test_df)
gbt_predictions.select('prediction', 'medv', 'features').show(5)

+------------------+----+--------------------+
|        prediction|medv|            features|
+------------------+----+--------------------+
| 31.80402021472451|32.2|[0.00906,90.0,2.9...|
| 33.35970866216509|35.4|[0.01311,90.0,1.2...|
|23.231048108369254|23.1|[0.0187,85.0,4.15...|
|  33.8906112621149|33.0|[0.01951,17.5,1.3...|
|50.417914041805176|42.3|[0.02177,82.5,2.0...|
+------------------+----+--------------------+
only showing top 5 rows



In [42]:
gbt_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="rmse")
rmse = gbt_evaluator.evaluate(gbt_predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

Root Mean Squared Error (RMSE) on test data = 4.43236


In [43]:
gbt_evaluator = RegressionEvaluator(
    labelCol="medv", predictionCol="prediction", metricName="r2")
r2 = gbt_evaluator.evaluate(gbt_predictions)
print("R2 Score on test data = %g" % r2)

R2 Score on test data = 0.769376
